# Project Title - Edit Me


## Data set selection

> In this section, you will need to provide the following information about the selected data set:
>
> - Source with a link: 
- Sleep Health and Lifestyle Dataset - https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
> - Fields: 
- Person ID, Gender, Age, Occupation, Sleep Duration, Quality of Sleep, Physical Activity Level, Stress Level, BMI Category, Blood Pressure, Heart Rate, Daily Steps, Sleep Disorder
> - License: 
- Public Domain

### Data set selection rationale

> Why did you select this data set?
> I selected this dataset for a few reasons
- 13 diverse fields that would make for great visualizations 
- Large data set (300+ records)
- I've used datasets from Kaggle before and have found it to be a great resource
 

### Questions to be answered

> Using statistical analysis and visualization, what questions would you like to be able answer about this dataset.
> This could include questions such as:
>
> - What is the relationship between X and Y variables?
> - What is the distribution of the variables?
> - What is the relationship between the variables and the target?
>   You will need to frame these questions in a way to show value to a stakeholder (i.e.why should we know about the relationship between X and Y variables?)
>
My questions to be answered:
- What factors most influence sleep? For example:

- Do men or women experience better sleep health?
- Do certain high stress careers imapct sleep health?
- What is the correlation between BMI and sleep health?
- What is the correlation between daily steps and sleep health?
- What is the correlation between heart health and sleep health? 
- What is the correlation between stress and quality of sleep?


### Visualization ideas

> Provide a few examples of what you plan to visualize to answer the questions you posed in the previous section. In this project, you will be producing 6-8 visualizations. You will also be producing an interactive chart using Plotly.
> Think about what those visualization could be: what are the variables used in the charts? what insights do you hope to gain from them?
> Potential visualization ideas:
- Histogram showing distribution of sleep length
- Scatter plot showing sleep length vs sleep quality
- Heatmap(s) of health metrics including BMI, steps, physical activity vs sleep duration
- Bar chart of stress level by occupation
- Bar chart showing quality of sleep by occupation 
- Bar chart(s) showing sleep disorders by gender
- Interactibe plotly chart showcasing occupation, stress, and quality of sleep


In [ ]:
# 🚀 Importing some libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns